<div>
    <h1 style="margin-top:1em;font-weight:700">ALGORITHM 1 (SPIKE SORTING)</h1>
    <br>
    <h3 style="margin-left:1em; margin-top:1em;font-weight:700">NAVIGATION BUTTONS</h3>
    <a href="#0" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#EEEEEE;float:left;margin-left:1em;margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em; ">
            0
        </center>
        <center style="font-weight:700; font-size:1em; ">
            ALGORITHM SETTINGS
        </center>
    </div>
    </a>
    <a href="#1" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#F0C38F;float:left;margin-left:1em;margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em; ">
            1
        </center>
        <center style="font-weight:700; font-size:1em; ">
            DATA READING AND NOISE FILTERING
        </center>
    </div>
    </a>
    <a href="#2" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#EE999F; float:left; margin-left:1em; margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em;">
            2
        </center>
        <center style="font-weight:700; font-size:1em; ">
            FILTERING
        </center>    
    </div>
    </a>
    <a href="#3" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#AECDAD; float:left; margin-left:1em; margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em;">
            3
        </center>
        <center style="font-weight:700; font-size:1em; ">
            THRESHOLDING AND SPIKE EXTRACTION
        </center>
    </div>
    </a>
    <a href="#4" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#7EC0EE; float:left; margin-left:1em; margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em;">
            4
        </center>
        <center style="font-weight:700; font-size:1em; ">
            FEATURE EXTRACTION (PCA)
        </center>
    </div>
    </a>
    <a href="#5" style="text-decoration:none; color:#000000">
    <div style="padding:1em; height:12em; width:12em; border-radius:12em;  background-color:#E0BBE4; float:left; margin-left:1em; margin-top:1em">
        <center style="margin-top:1.2em; font-weight:700; font-size:2em;">
            5
        </center>
        <center style="font-weight:700; font-size:1em; ">
            CLUSTERING
        </center>
    </div>
    </a>
</div>

<br><br><br><br><br>

<a id='0'></a>
<div style="background-color:#EEEEEE; font-size:1.5em;text-align:left;padding:0.7em;"><b>0) ALGORITHM SETTINGS</b></div>
<hr>

#### IMPORTING LIBRARIES

In [ ]:
from Alg1 import *
plt.rcParams.update(plt.rcParamsDefault)
pltstyle.use('seaborn-dark') #Comment this line if you want the white graphs

#### SAVING FILES OPTIONS
<p>Set the corresponding "save" option to True or False to create a folder on the desktop and save all the pictures and data</p>

In [ ]:
save_images = False
save_data = False

# Do you want the folder name to contain the date of its creation? (It is useful if you run the code multiple times)
time_name = False

#### READING FILES OPTIONS
<p>Set to True to read already created files. You have to set the directory of the files you want to read below in the code (see red checkpoints). <b>If it is the first time you run this algorithm set to False</b></p>

In [ ]:
read_data = False

#### INPUT-OUTPUT FOLDERS NAMES

<p>Set here the input and output folder location:</p>

In [ ]:
#Set the path to the file .h5 containing the signal
input_path = ""

#Set the name of the folder that will contain all the results
MEA = ""
cell_type = ""
output_folder_name = "Risultati " + MEA + " " + cell_type

In [ ]:
if (save_images == True) or (save_data == True):
    if time_name == True: 
        now = str(datetime.datetime.now())[:19]
        now = now.replace(":",".")
        desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
        output_path = desktop + "/" + output_folder_name + " " + now
        os.makedirs(output_path)
        print("A new folder named", output_folder_name + " " + now, "that will contain all the results has been created on your desktop")
    else:
        desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
        output_path = desktop + "/" + output_folder_name
        os.makedirs(output_path)
        print("\nA new folder named '" + output_folder_name + "' has been created on your desktop")
else:
    current_dir = None
    
if read_data == True:
    desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
    output_path = desktop + "/" + output_folder_name
    files_saved_path = output_path + "/Files Saved"
    
if save_data == True:
    os.makedirs(files_saved_path)

<hr><br><br><br><br><br>
<a id='1'></a>
<div style="background-color:#F0C38F; font-size:1.5em;text-align:left;padding:0.7em"><b>1) DATA READING + NOISE FILTERING</b></div> 
<hr>

<p>We plotted the useful information regarding the acquisition system (MEA) and we plotted a Power Spectral Density to see if there were any noise in the recordings. </p>

- We found noises in all datasets at low frequencies and decided to filter those out using a bandpassfilter.

- We also found some noise at certain frequencies (which depended on the dataset we were working with). Those frequencies were harmonics and we suppose that they were due to the acquisition system. We filtered them out using a notch filter.</p>

#### DATA READING

In [ ]:
channel_raw_data = RawData(input_path)
duration = channel_raw_data.recordings[0].duration_time.magnitude / 1000000   # from microseconds to seconds
timestamp = channel_raw_data.recordings[0].analog_streams[0].get_channel_sample_timestamps(0)[0]/1000000 # from microseconds to seconds
fs = 1/(timestamp[-1]/timestamp.shape[0])

print("\n\n\033[1;31;47mMEA INFORMATIONS\u001b[0m")
print(channel_raw_data.comment)
print(channel_raw_data.date)
print(channel_raw_data.clr_date)
print(channel_raw_data.date_in_clr_ticks)
print(channel_raw_data.file_guid)
print(channel_raw_data.mea_name)
print(channel_raw_data.mea_sn)
print(channel_raw_data.mea_layout)
print(channel_raw_data.program_name)
print(channel_raw_data.program_version)
print('Duration of the recording: ',round(duration,2),'seconds')
print('Sampling Frequency: ',round(fs,2),'Hz')


data = h5py.File(input_path,'r')
data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
labels = info_table['Label']
readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)

#Writing the info to a .txt file
if save_data == True:
    MEA_info = open(output_path + "/MEA_info.txt", "w")
    info = "MEA INFORMATION" + "\n" + "Dataset: " + input_path.split("/")[-1] + "\n" + "Duration of the recording: " + str(round(duration,2)) + ' seconds' + "\n" + "Sampling Frequency: " + str(round(fs,2)) +' Hz'
    MEA_info.write(info)
    MEA_info.close()

del data_readings, channel_raw_data, timestamp

#Plotting the unfiltered signal present on one channel 
plt.figure(figsize=(10,5))
plt.plot(readings[b'47'],label="Segnale dall'elettrodo 47", linewidth=1)
plt.title("Segnale non filtrato da un elettrodo di esempio")
plt.ylabel('Segnale [\u03BCV]')
plt.xlabel('Campioni [0.1ms]')
plt.grid()
plt.legend(loc='upper left')

#Saving image
if (save_images == True):
    file_name = output_path + "/unfiltered_signal_example.png"
    plt.savefig(file_name, dpi=300)

plt.show()

In [ ]:
#Working Directory of the Preliminar Filtering
if (save_images == True) or (save_data == True):
    current_dir = output_path + "/1. Data Reading and Preliminar Filtering"
    os.makedirs(current_dir)

In [ ]:
#PLOT UNFILTERED PSD

colormap = cm.get_cmap('hsv')
plt.figure(figsize=(12,12))



#First Plot
plt.subplot(2, 1, 1)
plt.title('Spettro di potenza (PSD)')
column = 0
win_len = 1024*2
overlap = 0.5*win_len
for electrode in tqdm(readings.columns):
    if electrode == b'Ref':
        plt.psd(readings[electrode],1024 ,Fs=fs,window=np.hamming(1024),noverlap=50, label=str(electrode, 'utf-8'), color = "#000000", linewidth=0.7)
    else:
        plt.psd(readings[electrode],1024 ,Fs=fs,window=np.hamming(1024),noverlap=50, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column], linewidth=0.7)
    plt.axis([0,5000,-20,52])
    column +=1
plt.legend(loc='upper right', ncol=8)
plt.ylabel('Spettro di Potenza [dB/Hz]')
plt.xlabel('Frequenza [Hz]')



#Second Plot
plt.subplot(2, 1, 2)
plt.title('Spettro di potenza (PSD) - Zoom alle basse frequenze')
column = 0
win_len = 1024*16
overlap = 0.5*win_len
for electrode in tqdm(readings.columns):
    if electrode == b'Ref':
        plt.psd(readings[electrode], win_len ,Fs=fs,window=np.hamming(win_len), noverlap=overlap, label=str(electrode, 'utf-8'), color = "#000000", linewidth=0.7)
    else:
        plt.psd(readings[electrode], win_len ,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column], linewidth=0.7)
    plt.axis([0,250,-20,52])
    column +=1
plt.legend(loc='upper right', ncol=8)
plt.ylabel('Spettro di Potenza [dB/Hz]')
plt.xlabel('Frequenza [Hz]')

#Saving image
if (save_images == True):
    file_name =current_dir + "/raw_psd.png"
    plt.savefig(file_name, dpi=300)

plt.show()

#### FILTERING NOISES

In [ ]:
#PREFILTERING - LOW FREQUENCIES
readings_rows = range(readings.shape[0])
reading_columns = range(readings.shape[1])
pre_filtered_readings = pd.DataFrame(data = 0, columns=readings.columns, index=readings_rows, dtype = "float32")

b, a = scipy.signal.butter(N = 8, Wn=200, btype='highpass', analog=False, output='ba', fs=fs)
for x in tqdm(range(readings.shape[1])):
    pre_filtered_readings.values[:,x] = scipy.signal.filtfilt(b, a, readings.values[:,x])



#Prefiltered plot - Low Frequencies removed
colormap = cm.get_cmap('hsv')
plt.figure(figsize=(12,12))
plt.subplot(2, 1, 1)
plt.title('Spettro di potenza (rumore a bassa frequenza rimosso)')
column = 0
win_len = 1024*2
overlap = 0.5*win_len
for electrode in tqdm(readings.columns):
    if electrode == b'Ref':
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = "#000000", linewidth=0.7)
    else:
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column], linewidth=0.7)
    plt.axis([0,5000,-20,52])
    column +=1
plt.ylabel('Spettro di Potenza [dB/Hz]')
plt.xlabel('Frequenza')
plt.legend(loc='upper right', ncol=8)



#Second Plot 
win_len = 1024*16
overlap = 0.5*win_len
plt.subplot(2, 1, 2)
plt.title('Spettro di potenza - Zoom alle basse frequenze (rumore a bassa frequenza rimosso)')
column = 0
for electrode in tqdm(readings.columns):
    if electrode == b'Ref':
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = "#000000", linewidth=0.7)
    else:
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column], linewidth=0.7)
    plt.axis([0,250,-20,52])
    column +=1
plt.ylabel('Spettro di Potenza [dB/Hz]')
plt.xlabel('Frequenza')
plt.legend(loc='upper right', ncol=8)


if (save_images == True):
    file_name =current_dir + "/freq_below_200_removed_psd.png"
    plt.savefig(file_name, dpi=300)

plt.show()

In [ ]:
#PREFILTERING - ADDITIONAL TWO PEAKS NOISES

detect_band2 = False
# True => calcola la seconda soglia indipendentemente dalla prima. Questo è utile se non si assume che il secondo rumore sia a frequenze multuiple del primo
# False => calcola la seconda banda come multiplo della prima. Potrebbe in generale funzionare meglio, specialmente se il rumore al secondo picco è molto poco intenso

percentage_increase_band1 = 0.3
percentage_increase_band2 = 0.3
#Le due percentuali di incremento delle bande da arrestare. Sono da scegliere in base ai grafici. 
#Se determinazione_banda_due è False, allora percentuale_incremento_2 non viene considerato

samples_moving_avarage = 200
#Determina lo "smooth" dei grafici della FFT. è utile se il rumore va su e giù rapidamente e rischia di superare la soglia in più punti

multiple_threshold1 = 2
multiple_threshold2 = 1.4
#Determina la soglia come multiplo del valor medio del modulo della FFT. 
#Se determinazione_banda_2 è False allora multiplo_soglia_2 non viene considerato

#Aggiungere second peak
a_1, b_1, a_2, b_2, second_peak = detect_ref_noise(ref=pre_filtered_readings[b'Ref'].values, m1 = percentage_increase_band1, m2=percentage_increase_band2, multiple_first_peak=multiple_threshold1, multiple_second_peak=multiple_threshold2, current_dir=current_dir, fs=fs, detect_second_peak= detect_band2, n=samples_moving_avarage, save=save_images)

In [ ]:
#FIRST PEAK DETECTED
print("Filtering the first peak:")
time.sleep(0.3)
for x in tqdm(range(readings.shape[1])):
    pre_filtered_readings.values[:,x] = scipy.signal.filtfilt(b=b_1, a=a_1, x=pre_filtered_readings.values[:,x])

#SECOND PEAK DETECTED (NOT ALWAYS PRESENT)  
if (second_peak == True):
    print("Filtering the second peak:")
    time.sleep(0.3)
    for x in tqdm(range(readings.shape[1])):
        pre_filtered_readings.values[:,x] = scipy.signal.filtfilt(b=b_2, a=a_2, x=pre_filtered_readings.values[:,x])

print("Plotting the graph:")
#Prefiltered plot - All noise removed
colormap = cm.get_cmap('hsv')
plt.figure(figsize=(12,12))
plt.subplot(2, 1, 1)
plt.title('Spettro di potenza (picchi di rumore rimossi)')
column = 0
win_len = 1024*2
overlap = 0.5*win_len
for electrode in tqdm(readings.columns):
    if electrode == b'Ref':
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = "#000000", linewidth=0.7)
    else:
        plt.psd(pre_filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column], linewidth=0.7)
    plt.axis([0,5000,-20,52])
    column +=1

plt.ylabel('Spettro di Potenza [db/Hz]')
plt.xlabel('Frequenza')
plt.legend(loc='upper right', ncol=8)

if (save_images == True):
    file_name = current_dir + "/all_noise_removed_psd.png"
    plt.savefig(file_name, dpi=300)

plt.show()

#### DIFFERENTIAL READING

In [ ]:
#Differential Reading
for electrode in pre_filtered_readings.columns:
    pre_filtered_readings[electrode] = pre_filtered_readings[electrode] - pre_filtered_readings[b'Ref']
    
pre_filtered_readings = pre_filtered_readings.drop([b'Ref'],axis = 1)

<hr>

<br><br><br><br><br>
<a id='2'></a>
<div style="background-color:#EE999F; font-size:1.5em;text-align:left;padding:0.7em"><b>2) FILTERING + CORRELATION MATRIX</b></div> 
<hr>

We first need to see the frequencies that define an action potential of a neuron. So we can plot the time domain signal vs its Short Time Fourier Transform (STFT)
<br>
We found the typical frequencies of 300Hz 3000Hz cited in various papers well fit our data. 
<br>
- Low frequencies are important for spike amplitude (especially for the slowest ones)
- High frequencies are important for the morphology of the spikes

<br>
We then tried the following filters implemented using zero-phase filtering:

1. FIR (Hamming window)
2. FIR (Blackman-Nuttall window)
3. IIR (Butterworth)

We decided to filter the signal with the Butterworth one (due to maximal flatness) between 300Hz and 3000Hz 

#### FILTERING

In [ ]:
#Working Directory of the Filtering + Differential Reading + Correlation Matrix
if (save_images == True) or (save_data == True):
    current_dir = output_path + "/2. Filtering"
    os.makedirs(current_dir)

In [ ]:
electrode_example = b'36' #Change the name here to change the electrode

t_initial = 20000 #Change the number here to select a different initial position
lenght = 600 #Change the number here to enlarge or reduce the x axis (time)

#Time domain signal
plt.figure(figsize=(15,15))
plt.subplot(2, 1, 1)
label = "Channel " + str(electrode_example, 'utf-8')
plt.plot(pre_filtered_readings[electrode_example].values, label=label)
plt.axis([t_initial,t_initial+lenght, -700, 400])
plt.ylabel('Segnale [\u03BCV]')
plt.xlabel('Campioni')
plt.grid()
plt.legend(loc="upper right")


#Short Time Fourier Transform
plt.rcParams.update(plt.rcParamsDefault)
plt.subplot(2, 1, 2)
f, t, Sxx = scipy.signal.stft(x = pre_filtered_readings[electrode_example].values[t_initial:t_initial+lenght], fs=fs, window='boxcar', nperseg=2**6, noverlap=None, nfft=2**9)
plt.pcolormesh(t*fs+t_initial, f, np.abs(Sxx))
plt.colorbar(orientation='horizontal')
plt.ylabel('Frequenza [Hz]')
plt.xlabel('Campioni')
plt.axis
plt.ylim([0,5000])

#Saving images
if (save_images == True):
    file_name = current_dir + "/freq_analysis_with_stft" + "_" + str(electrode_example) + ".png"
    plt.savefig(file_name, dpi=300)

plt.show()

pltstyle.use('seaborn-dark') #Comment this line if you want the white graphs from now on

In [ ]:
lowcut = 300
highcut = 3000

readings_rows = range(pre_filtered_readings.shape[0])
reading_columns = range(pre_filtered_readings.shape[1])
filtered_readings = pd.DataFrame(data = 0, columns=pre_filtered_readings.columns, index=readings_rows, dtype = "float32")

elab = pre_filtered_readings.values

order=8
for x in tqdm(range(filtered_readings.shape[1])):
    filtered_readings.values[:,x] = butter_bandpass_filter(elab[:,x], lowcut, highcut, fs, order=order)

del readings
del pre_filtered_readings
del elab

In [ ]:
plt.figure(figsize=(20,9))
plt.rcParams.update({'font.size': 20})
column = 0
colormap = cm.get_cmap('hsv')
for electrode in tqdm(filtered_readings.columns):
    if electrode == b'Ref':
        plt.psd(filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = "#000000")
    else:
        plt.psd(filtered_readings[electrode], win_len,Fs=fs,window=np.hamming(win_len),noverlap=overlap, label=str(electrode, 'utf-8'), color = colormap(np.linspace(0, 1, 60))[column])
    plt.axis([0,5000,-25,35])
    plt.title('Spettro di potenza')
    column +=1
plt.ylabel('Spettro di Potenza [dB/Hz]')
plt.xlabel('Frequenza')

plt.rcParams.update({'font.size': 15})
plt.legend(loc='upper right', ncol=10)

if (save_images == True):
    file_name = current_dir + "/filtered_psd.png"
    plt.savefig(file_name, dpi=300)

plt.show()

##### CORRELATION MATRIX AND DELETION OF HIGH CORRELATED CHANNELS

In [ ]:
corr_matrix = filtered_readings.iloc[:1000000,:].corr()
plt.figure(figsize=(30,30))
sns.heatmap(corr_matrix, annot=False, yticklabels = True)
plt.title('Correlation Matrix')

if (save_images == True):
    file_name = current_dir + "/correlation_matrix_before.png"
    plt.savefig(file_name, dpi=300)

plt.show()

In [ ]:
max_corr = 0.85

correlated_channels = set()

for column in corr_matrix.columns:
    for row in corr_matrix.index:
        if row != column and corr_matrix[row][column] > max_corr:
            correlated_channels.add(row)

print("\nThe following", len(correlated_channels),"channels are correlated: ", correlated_channels,"\nWith a correlation over:", max_corr*100, "%\n")

absolute_mean = dict()
for electrode in correlated_channels:
    absolute_mean[electrode] = np.mean(abs(filtered_readings[electrode].values))
    
#Keeping only the channel that has maximum absolute value
maximum_value = float(0)
maximum_name = ""
for electrode in absolute_mean:
    if absolute_mean[electrode] > maximum_value:
        maximum_value = absolute_mean[electrode]
        maximum_name = electrode

correlated_channels.remove(maximum_name)

#Deleting all the other channels from the correlation matrix
corr_matrix = corr_matrix.drop(labels = correlated_channels, axis=0)
corr_matrix = corr_matrix.drop(labels = correlated_channels, axis=1)

#Deleting all the other channels from the filtered readings
filtered_readings = filtered_readings.drop(columns = correlated_channels)

plt.figure(figsize=(30,30))
sns.heatmap(corr_matrix, annot=False, yticklabels = True)
plt.title('Correlation Matrix')

if (save_images == True):
    file_name = current_dir + "/correlation_matrix_after.png"
    plt.savefig(file_name, dpi=300)

plt.show()

<hr><br><br><br><br><br>
<a id='3'></a>
<div style="background-color:#AECDAD; font-size:1.5em;text-align:left;padding:0.7em"><b>3) THRESHOLDING + SPIKE EXTRACTION</b></div> 
<hr>

<p>We tried two ways of calculating the threshold:</p>

1. Standard Deviation (SD)
2. Median Absolute Deviation (MAD)

<p>We opted for the estimator which better identified the noise variance, which is the MAD. The Median Absolute Deviation is indeed less sensible to the presence of outliers (i.e. spikes) and this is useful as we need to estimate only the background noise amplitude (without the presence of spikes). The real threshold is then a multiple of the chosen estimator.</p>
<p>We then extracted the signal which exceeded the threshold which is presumably the spikes<br>
At the end of this section there is a function that allows us to see the number of spikes detected as a function of the multiple of the chosen threshold</p>

In [ ]:
#Working Directory of the Thresholding + Spike Extraction
if (save_images == True) or (save_data == True):
    current_dir = output_path + "/3. Thresholding + Spike Extraction"
    os.makedirs(current_dir)

#### THRESHOLD

In [ ]:
#You can choose "MAD" and "STD"
threshold_method = "MAD"

#Multiple
n = 4

In [ ]:
if threshold_method == "MAD":
    deviation_list = pd.DataFrame(data = 0, columns=filtered_readings.columns, index=range(1))
    threshold = pd.DataFrame(data = 0, columns=filtered_readings.columns, index = range(1))
    for electrode in tqdm(filtered_readings.columns):
        deviation_list[electrode] = scipy.stats.median_absolute_deviation(filtered_readings[electrode].values)
        threshold[electrode] = (deviation_list[electrode])*n
    
elif threshold_method == "STD":
    deviation_list = pd.DataFrame(data = 0, columns=filtered_readings.columns, index=range(1))
    threshold = pd.DataFrame(data = 0, columns=filtered_readings.columns, index = range(1))
    for electrode in tqdm(filtered_readings.columns):
        deviation_list[electrode] = np.std(filtered_readings[electrode].values)
        threshold[electrode] = (np.std(filtered_readings[electrode].values))*n
        
else:
    raise Exception("The variable named 'threshold_method' must be either 'MAD' or 'STD'")

#### SPIKE EXTRACTION

In [ ]:
filtered_readings.columns

In [ ]:
legend = pd.DataFrame(data = 0, columns=filtered_readings.columns, index=["corresponding number"])
crossings = []
crosscorr_indexes = []
indexes = []
extracted = []


electrode_list = filtered_readings.columns
    
i = 0
for electrode in tqdm(filtered_readings.columns):


    legend[electrode] = i
    sgn = filtered_readings[electrode].values
    thr = float(threshold[electrode].values[0])

    #Detect crossing of the threshold
    crossings.append(DetectCrossings(signal = sgn, threshold = thr, fs = fs))
    
    print("Il canale " + str(electrode) + " ha: " + str(len(crossings[i])) + " spike")
    
    #Align the spikes to crosscorrelation
    crosscorr_indexes.append(AlignToCrosscorrelation(signal = sgn, given_indexes = np.array(crossings[i], dtype=int), fs = fs))
    
    #Extract the spikes and their indexes
    spks, idx = ExtractWaveform(signal = sgn, indexes = np.array(crosscorr_indexes[i], dtype=int), fs = fs)
    #Save the spikes
    extracted.append(spks)
    #Save their indexes
    indexes.append(idx)
    
    i+=1

del crossings, crosscorr_indexes, spks, idx, sgn, thr

In [ ]:
#Number of spikes detected as a function of the multiple of the threshold

ThresholdVariation(filtered_readings=filtered_readings, save=save_images, current_dir=current_dir, aligned_indexes=indexes, deviation_list=deviation_list, fs=fs)


<br><br><hr>
<h3 style="color:red">CHECKPOINT</h3>

<h4 style="color:black">Saving Waveforms</h4>

In [ ]:
if save_data == True:  
    file_name = files_saved_path + "/ExtractedWaveforms.h5"
    print("Saving path: " + file_name)
    
    decoded_electrode_list = []
    for electrode in electrode_list:
        decoded_electrode_list.append(int(electrode.decode()))
        
    keys = np.array(range(len(extracted)))

    hf = h5py.File(file_name, 'w')
    
    hf.create_dataset("electrode_list", data=decoded_electrode_list)
    hf.create_dataset("keys", data=keys)
    for i in range(len(extracted)):
        data_to_save = extracted[i]
        data_name = str(i)
        hf.create_dataset(data_name, data=data_to_save)
        
    hf.close()
    
    print("Waveforms saved!")

<h4 style="color:black">Saving Indexes</h4>

In [ ]:
if save_data == True:  
    file_name = files_saved_path + "/ExtractedIndexes.h5"
    print("Saving path: " + file_name)
    
    decoded_electrode_list = []
    for electrode in electrode_list:
        decoded_electrode_list.append(int(electrode.decode()))
        
    keys = np.array(range(len(indexes)))

    hf = h5py.File(file_name, 'w')
    
    hf.create_dataset("electrode_list", data=decoded_electrode_list)
    hf.create_dataset("keys", data=keys)
    for i in range(len(indexes)):
        data_to_save = indexes[i]
        data_name = str(i)
        hf.create_dataset(data_name, data=data_to_save)
        
    hf.close()
    
    print("Indexes saved!")

<h4 style="color:black">Saving Legend</h4>

In [ ]:
if (save_data == True):
    
    #Decoding from bytes to str
    decoded_electrodes = []
    for electrode in legend.columns:
        decoded_electrodes.append(electrode.decode())

    legend.columns = decoded_electrodes
    del decoded_electrodes
    
    file_name = files_saved_path + "/legend.h5"
    legend.to_hdf(file_name, key='data', mode='w')
    
    #Re-Encoding from str to bytes
    encoded_electrodes = []
    for electrode in legend.columns:
        encoded_electrodes.append(electrode.encode(encoding='utf-8'))
        
    legend.columns = encoded_electrodes

<h4 style="color:black">Saving (Sampling Frequency) and (Number of Samples)</h4>

In [ ]:
if save_data == True:
    time_vector = [fs, filtered_readings.shape[0]]
    file_name = files_saved_path + "/time_vector"
    np.savez(file_name, time_vector)
    print("Saved!")

<br><br><br>
<h4 style="color:black">Loading Waveforms</h4>
<p>You have to add the path to the file containing ExtractedWaveforms.h5 of the correct dataset</p>

In [ ]:
files_saved_path = "/Users/lucasubitoni/OneDrive - Politecnico di Milano/3° Anno/2° Semestre/Progetto/Tesi scritta/Risultati Algoritmo 1/Risultati MIP 4 LRKK2/Files Saved"

In [ ]:
if read_data == True:
    extracted = []
    
    file_name = files_saved_path + "/ExtractedWaveforms.h5"
    print("Loading path: " + file_name)
    hf = h5py.File(file_name, 'r')
    
    electrode_list = []
    for electrode in np.array(hf.get("electrode_list")):
        electrode_list.append(str(electrode).encode(encoding='utf-8'))
    
    for i in np.array(hf.get("keys")):
        data_i = hf.get(str(i))
        data_i = np.array(data_i)
        extracted.append(data_i)

    print("Waveforms loaded!")

<h4 style="color:black">Loading Indexes</h4>
<p>You have to add the path to the file containing ExtractedIndexes.h5 of the correct dataset</p>

In [ ]:
if read_data == True:
    indexes = []
    
    file_name = files_saved_path + "/ExtractedIndexes.h5"
    print("Loading path: " + file_name)
    hf = h5py.File(file_name, 'r')
    
    electrode_list = []
    for electrode in np.array(hf.get("electrode_list")):
        electrode_list.append(str(electrode).encode(encoding='utf-8'))
    
    for i in np.array(hf.get("keys")):
        data_i = hf.get(str(i))
        data_i = np.array(data_i)
        indexes.append(data_i)

    print("Indexes loaded!")

<h4 style="color:black">Loading Legend</h4>
<p>You have to add the path to the file containing Legend.h5 of the correct dataset</p>

In [ ]:
if (read_data == True):
    file_name = files_saved_path + "/legend.h5"
    print("Loading path: " + file_name)
    legend = pd.read_hdf(file_name, 'data')
    
    #Encoding from str to bytes
    encoded_electrodes = []
    for electrode in legend.columns:
        encoded_electrodes.append(electrode.encode(encoding='utf-8'))

    legend.columns = encoded_electrodes
    del encoded_electrodes
    
print("Legend loaded!")

<hr><br><br><br><br><br>
<a id='4'></a>
<div style="background-color:#7EC0EE; font-size:1.5em;text-align:left;padding:0.7em"><b>4) EXTRACT SPIKE CHARACTERISTICS (PCA)</b></div> 
<hr>

We used Principal Component Analysis (PCA) to find the 3 most important characteristics that defined the spikes detected by a single electrode. We investigated two main ways of detecting and extracting those characteristics:
1. Morphologic characteristics: independent on the previous alignment method; the spike has to be cut entirely but not perfectely aligned with others. (those characteristics are maximum/minimum voltage, the maxiumum/minimum derivative and so on...) 
2. Voltage dependent characteristics: dependent on the previous alignment method; the characteristics are the voltages at each point

We chose to use the voltage dependent characteristics as the crosscorrelation algorithm is quite robust
<br>

In [ ]:
#Working Directory of the Extract Characteristics
if (save_images == True) or (save_data == True):
    current_dir = output_path + "/4. PCA"
    os.makedirs(current_dir)

In [ ]:
plt.clf() #To clear the cache of matplotlib
principal_components = []

for electrode in electrode_list:
    print("\n\nELECTRODE: ", electrode)
    principal_components.append(PerformPCA(x=extracted[legend[electrode].values[0]], current_dir=current_dir, electrode_name=electrode, n=3, show=True, save=save_images))
    

<hr><br><br><br><br><br>
<a id='5'></a>
<div style="background-color:#E0BBE4; font-size:1.5em;text-align:left;padding:0.7em"><b>5) CLUSTERING</b></div> 
<hr>

We tried three main ways of clustering neurons together.
1. DBSCAN
2. KMEANS
3. HIERARCHICAL

In [ ]:
#Working Directory of the Clustering
if (save_images == True) or (save_data == True):
    current_dir = output_path + "/5. Clustering"
    os.makedirs(current_dir)

In [ ]:
#You can choose between "DBSCAN" and "KMEANS" and "HIERARCHICAL" or "ALL" to run them all and compare the outputs

clustering_method = "HIERARCHICAL"

In [ ]:
plt.clf() #To clear the cache of matplotlib
points = len(extracted[0][0])

# DBSCAN===============================================================================================
if (clustering_method == "DBSCAN") or (clustering_method == "ALL"):
    
    #Directory of the DBSCAN
    if (save_images == True) or (save_data == True):
        DBSCAN_dir = current_dir + "/DBSCAN"
        os.makedirs(DBSCAN_dir)
    else:
        DBSCAN_dir = None

    DBSCAN_cluster = []
    for electrode in tqdm(electrode_list):
        print("\n\nELECTRODE: ",electrode)
        DBSCAN_cluster.append(perform_pca_DBSCAN(extracted[legend[electrode].values[0]],np.array(indexes[legend[electrode].values[0]]),
                                      fs = fs,n_comp=3, current_dir = DBSCAN_dir, electrode=electrode,points=points, distanza = 1, punti_min = 50, save=save_images, ignorare = 500))

    #Removal of possible empty clusters
    for i, electrode in enumerate(DBSCAN_cluster):
        if electrode == []:
            DBSCAN_cluster[i] = None
            
    


# KMEANS===============================================================================================
elif (clustering_method == "KMEANS") or (clustering_method == "ALL"):
    
    #Directory of the HIERARCHICAL
    if (save_images == True) or (save_data == True):
        KMEANS_dir = current_dir + "/KMEANS"
        os.makedirs(KMEANS_dir)
    else:
        KMEANS_dir = None

    KMEANS_cluster = []
    for electrode in tqdm(electrode_list):
        print("\n\nELECTRODE: ",electrode)
        KMEANS_cluster.append(perform_pca_KMEANS(cutouts=extracted[legend[electrode].values[0]], spike_list=np.array(indexes[legend[electrode].values[0]]), points=points, fs=fs, n_comp=3, electrode=electrode, current_dir=KMEANS_dir,  min_silhouette_score=0.275, ignorare=500, centroids=False, save=False))

    #Removal of possible empty clusters
    for i, electrode in enumerate(KMEANS_cluster):
        if electrode == []:
            KMEANS_cluster[i] = None

            
            
            
        
# HIERARCHICAL=========================================================================================
elif (clustering_method == "HIERARCHICAL") or (clustering_method == "ALL"):
    
    #Directory of the HIERARCHICAL
    if (save_images == True) or (save_data == True):
        HIERARCHICAL_dir = current_dir + "/HIERARCHICAL"
        os.makedirs(HIERARCHICAL_dir)
    else:
        HIERARCHICAL_dir = None

    HIERARCHICAL_cluster = []
    for electrode in tqdm(electrode_list):
        print("\n\nELECTRODE: ",electrode)
        HIERARCHICAL_cluster.append(perform_pca_HIERARCHICAL(cutouts=extracted[legend[electrode].values[0]], spike_list=np.array(indexes[legend[electrode].values[0]]), points=points, fs=fs, n_comp=3, electrode=electrode, current_dir=HIERARCHICAL_dir, min_silhouette_score=0.275, ignorare=500, centroids=False, save=save_images))

    #Removal of possible empty clusters
    for i, electrode in enumerate(HIERARCHICAL_cluster):
        if electrode == []:
            HIERARCHICAL_cluster[i] = None
            
    

            
#______________________________________________________________________________________________________EXCEPTION
else:
    raise Exception("The options are 'DBSCAN', 'KMEANS', 'HIERARCHICAL' or alternatively you can run them all by using the word 'ALL'")

<br><br>
<h3 style="color:black">SAVING CLUSTERED INDEXES</h3>

In [ ]:
if save_data == True: 

    if (clustering_method == "DBSCAN") or (clustering_method == "ALL"):
        file_name = files_saved_path + "/NeuronIndexesDBSCAN"
        np.savez(file_name, DBSCAN_cluster)
        print("DBSCAN clustered indexes saved!")
        
    elif (clustering_method == "KMEANS") or (clustering_method == "ALL"):
        file_name = files_saved_path + "/NeuronIndexesKMEANS"
        np.savez(file_name, KMEANS_cluster)
        print("KMEANS clustered indexes saved!")
        
    elif (clustering_method == "HIERARCHICAL") or (clustering_method == "ALL"):
        file_name = files_saved_path + "/NeuronIndexesHIERARCHICAL"
        np.savez(file_name, HIERARCHICAL_cluster)
        print("HIERARCHICAL clustered indexes saved!")
        
    else:
        raise Exception("The options are 'DBSCAN', 'KMEANS', 'HIERARCHICAL' or alternatively you can run them all by using the word 'ALL'")
        
print("Algorithm 1 ENDED")

<br><br><br><br><br><br><br><br>

#### EXTRA (PLOT A SPECIFIC CHANNEL)

In [ ]:
plt.clf()
#Use this cell of code to plot the spikes detected by a specific channel

electrode_example = b'36' #Change the name here to change electrode

spike_count = 1 #Change the number here to increase or decrase the spikes to draw. 
                    #To see all spikes use: len(extracted[legend[electrode_example].values[0]])

#_________________________________________________________________________________________________________

if len(extracted[legend[electrode_example].values[0]]) < spike_count:
    spike_count = len(extracted[legend[electrode_example].values[0]])

#Plot
plt.figure(figsize=(20,12))
plt.rcParams.update({'font.size': 25})
plt.xlabel('Tempo [0.1ms]')
plt.ylabel('Tensione [\u03BCV]')
i=0
while (i<spike_count):
    plt.plot(extracted[legend[electrode_example].values[0]][i+428], color="#000000", alpha=0.3)
    i += 1
plt.axhline(threshold[electrode_example].values, 0,1, color="#FF0000")
plt.axhline(-threshold[electrode_example].values, 0,1, color="#FF0000")
title = str("First ") + str(i) + str(" spike aligned and extracted from electrode ") + str(electrode_example)
#plt.title(title)
plt.grid()

if (save_images == True):
    file_name = current_dir + "/spike_from_" + str(electrode_example) + ".png"
    plt.savefig(file_name, dpi=300)

plt.show()

#### EXTRA SEE THE EFFECT OF THE BURSTING

In [ ]:
value_in = 74884
value_fin = 83836

#Plotting the filtered signal present on one channel to find bursts
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 25})
plt.plot(filtered_readings[b'37'], linewidth=1.5, label="Canale 37 MIP4")
plt.ylabel('Segnale [\u03BCV]')
plt.xlabel('Campioni [0.1ms]')
plt.grid(lw=2)
plt.axis([value_in-7000, value_fin+7000, -4000, 6000])
plt.axvline(value_in, color="red", alpha=0.3, lw=2)
plt.axvline(value_fin, color="red", alpha=0.3, lw=2)
plt.legend(loc='upper right')
plt.show()

In [ ]:
value_in = 80000
value_fin = 84000

#Plotting the filtered signal present on one channel to find bursts
plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 25})
plt.plot(filtered_readings[b'37'], linewidth=1.5, label="Canale 37 MIP4")
plt.ylabel('Segnale [\u03BCV]')
plt.xlabel('Campioni [0.1ms]')
plt.grid(lw=2)
plt.axis([value_in-45000, value_fin+55000, -4000, 6000])
#plt.axvline(value_in, color="red", alpha=0.3, lw=2)
#plt.axvline(value_fin, color="red", alpha=0.3, lw=2)
plt.legend(loc='upper right')
plt.show()